# pandas

In [30]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

## Reading files using relative path 

In [15]:
cwd = Path.cwd()  # find the current working directory (cwd)
data_path = ((cwd).resolve().parents[0] / "./data/").resolve() # determine data path

In [16]:

df = pd.read_csv(os.path.join(data_path, "sample-sales-data.csv"))
df.head()

,store,product,sales_date,sales_qty,sales_rev
0,1,29790057,2020-03-19,4,42.0
1,2,29790057,2020-03-18,1,10.5
2,3,29790057,2020-03-16,8,84.0
3,4,29790057,2020-03-16,15,157.5
4,5,29790057,2020-03-17,28,294.0


In [17]:
print(df.dtypes)

store           int64
product         int64
sales_date     object
sales_qty       int64
sales_rev     float64
dtype: object


### parse_dates and dtypes

In [22]:
df = pd.read_csv(
    os.path.join(data_path, "sample-sales-data.csv"),
    parse_dates=["sales_date"]
)
df.dtypes

store                  int64
product                int64
sales_date    datetime64[ns]
sales_qty              int64
sales_rev            float64
dtype: object

### usecols

In [20]:
df = pd.read_csv(
    os.path.join(data_path, "sample-sales-data.csv"),
    usecols=["store","product","sales_qty"]
)
df.dtypes

store        int64
product      int64
sales_qty    int64
dtype: object

### nrows and shape

In [24]:
df = pd.read_csv(
    os.path.join(data_path, "sample-sales-data.csv"), nrows=100
)
df.shape

(100, 5)

## value_counts

In [25]:
df["store"].value_counts()

3    29
4    22
2    19
5    16
6     8
1     6
Name: store, dtype: int64

## astype

In [26]:
df.dtypes

store           int64
product         int64
sales_date     object
sales_qty       int64
sales_rev     float64
dtype: object

In [28]:
df["sales_date"] = df["sales_date"].astype("datetime64[ns]")
df.dtypes

store                  int64
product                int64
sales_date    datetime64[ns]
sales_qty              int64
sales_rev            float64
dtype: object

In [29]:
df = df.astype(
    {"store":"category","sales_qty":"float"}
)
df.dtypes

store               category
product                int64
sales_date    datetime64[ns]
sales_qty            float64
sales_rev            float64
dtype: object

## isna

In [31]:
customer = pd.DataFrame({
    "id": [11001, 11102, 11005, 11107, 11010],
    "name": ["John","Jane","Matt","Ashley","Emily"],
    "salary": [75000, 72000, np.nan, 76000, np.nan],
    "city": ["Houston","Dallas","San Antonio","Houston", np.nan],
    "start_date": ["2020-10-01","2021-11-10","2019-05-20",
                   "2019-07-19", np.nan]
})
customer

,id,name,salary,city,start_date
0,11001,John,75000.0,Houston,2020-10-01
1,11102,Jane,72000.0,Dallas,2021-11-10
2,11005,Matt,NaN,San Antonio,2019-05-20
3,11107,Ashley,76000.0,Houston,2019-07-19
4,11010,Emily,NaN,NaN,NaN


In [32]:
customer.isna().sum()

id            0
name          0
salary        2
city          1
start_date    1
dtype: int64

In [33]:
customer.isna().sum(axis=1) # for rows (axis=1)

0    0
1    0
2    1
3    0
4    3
dtype: int64

## dropna

In [34]:
# drop any row that does not have at least 3 non-missing values.
customer.dropna(axis=0, thresh=3, inplace=True)
customer

,id,name,salary,city,start_date
0,11001,John,75000.0,Houston,2020-10-01
1,11102,Jane,72000.0,Dallas,2021-11-10
2,11005,Matt,NaN,San Antonio,2019-05-20
3,11107,Ashley,76000.0,Houston,2019-07-19


## fillna

In [35]:
customer["salary"].fillna(customer["salary"].mean(), inplace=True)
customer

,id,name,salary,city,start_date
0,11001,John,75000.000000,Houston,2020-10-01
1,11102,Jane,72000.000000,Dallas,2021-11-10
2,11005,Matt,74333.333333,San Antonio,2019-05-20
3,11107,Ashley,76000.000000,Houston,2019-07-19


## notna

In [36]:
customer = pd.DataFrame({
    "id": [11001, 11102, 11005, 11107, 11010],
    "name": ["John","Jane","Matt","Ashley","Emily"],
    "salary": [75000, 72000, np.nan, 76000, np.nan],
    "city": ["Houston","Dallas","San Antonio","Houston", np.nan],
    "start_date": ["2020-10-01","2021-11-10","2019-05-20",
                   "2019-07-19", np.nan]
})
customer

,id,name,salary,city,start_date
0,11001,John,75000.0,Houston,2020-10-01
1,11102,Jane,72000.0,Dallas,2021-11-10
2,11005,Matt,NaN,San Antonio,2019-05-20
3,11107,Ashley,76000.0,Houston,2019-07-19
4,11010,Emily,NaN,NaN,NaN


In [37]:
customer[customer["salary"].notna()]

,id,name,salary,city,start_date
0,11001,John,75000.0,Houston,2020-10-01
1,11102,Jane,72000.0,Dallas,2021-11-10
3,11107,Ashley,76000.0,Houston,2019-07-19


## groupby

In [38]:
df.groupby("store")["sales_qty"].sum()

store
1     30.0
2     67.0
3    126.0
4     60.0
5     91.0
6     10.0
Name: sales_qty, dtype: float64

In [45]:
df_agg = df.groupby("store").agg(
    total_sales = ("sales_qty","sum")
)
df_agg

,total_sales
store,
1,30.0
2,67.0
3,126.0
4,60.0
5,91.0
6,10.0


## assign

In [49]:
df_agg.assign(
    more_than_fifty = np.where(df_agg["total_sales"] > 50, 1, 0)
)

,total_sales,more_than_fifty
store,,
1,30.0,0
2,67.0,1
3,126.0,1
4,60.0,1
5,91.0,1
6,10.0,0


## Credits
https://towardsdatascience.com/i-have-been-using-pandas-for-3-years-here-are-the-8-functions-i-use-the-most-4e54f4db5656

https://github.com/SonerYldrm/datasets